In [ ]:
import cv2
import numpy as np

image_pts = []

def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN and len(image_pts) < 10:
        image_pts.append((x, y))
        print(f"Point {len(image_pts)}: ({x}, {y})")
        if len(image_pts) == 10:
            cv2.destroyAllWindows()

# Load your image
img = cv2.imread('../Assets/house1.png')
if img is None:
    raise FileNotFoundError("Could not load house1.png")
img0 = cv2.imread('../Assets/0.png')
if img0 is None:
    raise FileNotFoundError("Could not load 0.png")

# Create window and set callback
cv2.namedWindow('Click 10 points', cv2.WINDOW_AUTOSIZE)
cv2.setMouseCallback('Click 10 points', click_event)

# Display image and collect 10 points
while True:
    cv2.imshow('Click 10 points', img)
    cv2.imshow('Referring image', img0)
    # Small wait to process GUI events
    key = cv2.waitKey(1) & 0xFF
    if len(image_pts) >= 10 or key == ord('q'):
        # Close window and give GUI time to update
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        break

image_pts = np.array(image_pts, dtype=float)

Point 1: (285, 408)
Point 2: (414, 267)
Point 3: (483, 274)
Point 4: (609, 61)
Point 5: (750, 215)
Point 6: (575, 482)
Point 7: (411, 58)
Point 8: (680, 337)
Point 9: (277, 245)
Point 10: (734, 393)
Selected image points (x, y):
[[285. 408.]
 [414. 267.]
 [483. 274.]
 [609.  61.]
 [750. 215.]
 [575. 482.]
 [411.  58.]
 [680. 337.]
 [277. 245.]
 [734. 393.]]


In [2]:
image_pts


array([[285., 408.],
       [414., 267.],
       [483., 274.],
       [609.,  61.],
       [750., 215.],
       [575., 482.],
       [411.,  58.],
       [680., 337.],
       [277., 245.],
       [734., 393.]])

In [4]:
# save points in txt file
with open('../Assets/image_pts.txt', 'w') as f:
    for pt in image_pts:
        f.write(f"{pt[0]} {pt[1]}\n")